In [ ]:
import numpy as np
import pandas as pd

In [ ]:
y = pd.read_parquet("../data/energy.parquet").asfreq("h").loc["2024":]
y.info()

In [ ]:
y

In [ ]:
# simulate some exogenous variables
X = pd.DataFrame(index=y.index)
X["rain"] = np.random.rand(len(X)) * 10
X.head()